Treinaremos nosso modelo ajustando os pesos várias vezes para fazer previsões cada vez melhores, utilizando uma técnica de otimização chamada gradiente de descida.

## Dados de treinamento

In [1]:
import numpy as np
import torch

Podemos representar os dados de treinamento utilizando duas matrizes: `inputs` e `targets`, cada uma com uma linha por observação e uma coluna por variável.

In [2]:
# Input (temperatura, chuva, umidade)
inputs = np.array(
    [[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]],
    dtype="float32",
)

In [3]:
# Targets (maçãs, laranjas)
targets = np.array(
    [[56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype="float32"
)

In [4]:
# Converte os inputs e targets para tensores
inputs_t = torch.from_numpy(inputs)
targets_t = torch.from_numpy(targets)

print(inputs)
print(targets)

[[ 73.  67.  43.]
 [ 91.  88.  64.]
 [ 87. 134.  58.]
 [102.  43.  37.]
 [ 69.  96.  70.]]
[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]


Os pesos e os viéses também podem ser representados por matrizes, inicializados como valores aleatórios por `randn`. A primeira linha de `w` e o primeiro elemento de `b` são usados para predizer a primeira variável-alvo.

In [5]:
# Pesos e bias
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-9.6039e-01, -1.1863e-03,  6.3681e-01],
        [-1.3369e+00,  2.6440e-01,  2.2328e-01]], requires_grad=True)
tensor([-1.4243,  0.9292], requires_grad=True)


In [6]:
def model(x):
    return x @ w.t() + b

In [7]:
model(inputs_t)

tensor([[ -44.2298,  -69.3488],
        [ -48.1689,  -83.1718],
        [ -48.2027,  -67.0013],
        [ -75.8736, -115.8043],
        [ -23.2288,  -50.3051]], grad_fn=<AddBackward0>)

A matriz obtida pela operação `x @ w.t() + b` é a predição do modelo para as variáveis alvos 

In [8]:
# Gerando previsões
preds = model(inputs_t)
print(preds)

tensor([[ -44.2298,  -69.3488],
        [ -48.1689,  -83.1718],
        [ -48.2027,  -67.0013],
        [ -75.8736, -115.8043],
        [ -23.2288,  -50.3051]], grad_fn=<AddBackward0>)


In [9]:
# Comparando com os targets
print(targets_t)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Função de perda

Agora que temos um modelo, podemos melhorá-lo.

Precisamos, antes desse passo, encontrar uma forma de saber se a predição foi além ou de menos. Podemos comparar o alvo com a previsão realizada com os seguintes meios:

* Calculando a diferença entre as duas matrizes
* Elevar todos os elementos ao quadrado para remover os valores negativos
* Calcular a média dos elementos resultantes

O resultado é um único valor, conhecido como MSE (mean square error) -- Erro Quadrático Médio

In [10]:
# Erro quadrático médio
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [11]:
# Calculando o erro
loss = mse(preds, targets_t)
print(loss)

tensor(22555.1523, grad_fn=<DivBackward0>)


In [12]:
# Gradientes
print(f'Gradiente de loss: {loss.backward()}')
print(f'Gradiente de w: {w.grad}')

Gradiente de loss: None
Gradiente de w: tensor([[-10462.1367, -11362.7910,  -6946.3579],
        [-14320.0586, -15033.5059,  -9376.8359]])


## Ajustando pesos e bias para reduzir a perda

O objetivo de todo treinamento de modelo é diminuir a perda.

Se o grandiente é positivo: 

* aumentar o valor do peso, aumentará o valor da perda.
* diminuir o valor do peso, diminuirá o valor da perda

In [13]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [14]:
# Verificamos se as perdas diminuiram
preds = model(inputs_t)
loss = mse(preds, targets_t)
print(loss)

tensor(15218.1172, grad_fn=<DivBackward0>)


In [15]:
w.grad.zero_()
b.grad.zero_()

print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Treinamento utilizando descida do gradiente

Como visto, reduzimos a perda para melhorar o modelo usando algoritmos de descida de gradiente. Para treinar modelos seguindo essa estratégia, precisamos:

1. Gerar predições
2. Calcular a perda
3. Calcular os gradientes
4. Ajustar os pesos substraindo por uma pequena fração do gradiente
5. Redefinir os gradientes a zero

In [16]:
# Geramos predições
preds = model(inputs_t)
print(preds)

tensor([[-25.9912, -44.7890],
        [-24.2022, -50.9082],
        [-19.8444, -28.9577],
        [-57.7449, -91.2623],
        [ -0.2380, -19.4266]], grad_fn=<AddBackward0>)


In [17]:
# Calculamos a perda
loss = mse(preds, targets_t)
print(loss)

tensor(15218.1172, grad_fn=<DivBackward0>)


In [18]:
# Computamos os gradientes
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -8579.1221,  -9339.2451,  -5697.7505],
        [-11785.5498, -12313.0684,  -7697.4321]])
tensor([-101.8041, -139.0687])


In [19]:
# Ajustamos o peso
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

    w.grad.zero_()
    b.grad.zero_()

In [20]:
print(w)
print(b)

tensor([[-0.7700,  0.2058,  0.7632],
        [-1.0758,  0.5379,  0.3940]], requires_grad=True)
tensor([-1.4220,  0.9323], requires_grad=True)


In [21]:
# Calculamos a perda
preds = model(inputs_t)
loss = mse(preds, targets_t)
print(loss)

tensor(10273.5918, grad_fn=<DivBackward0>)


## Treinamento por épocas

Para reduzirmos ainda mais a perda, podemos repetir esse processo por várias vezes (épocas). Vamos treinar por 100 épocas

In [22]:
# Treinamento por 100 epochs
for i in range(800):
    preds = model(inputs_t)
    loss = mse(preds, targets_t)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [23]:
# Calculamos a perda
preds = model(inputs_t)
loss = mse(preds, targets_t)
print(loss)

tensor(2.4225, grad_fn=<DivBackward0>)


In [24]:
# Verificamos as predições com os targets
preds, targets

# Registrando o loss de 8.6:
# (tensor([[ 57.3090,  69.2779],
#          [ 82.1458,  99.8025],
#          [118.6023, 136.5943],
#          [ 21.4513,  29.5221],
#          [101.6896, 122.3019]], grad_fn=<AddBackward0>),
#  array([[ 56.,  70.],
#         [ 81., 101.],
#         [119., 133.],
#         [ 22.,  37.],
#         [103., 119.]], dtype=float32))

(tensor([[ 56.7901,  70.5273],
         [ 83.5668,  99.8954],
         [116.1986, 134.3726],
         [ 20.4441,  37.2765],
         [104.3783, 117.8230]], grad_fn=<AddBackward0>),
 array([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]], dtype=float32))

# Regressão linear utilizando ferramentas do PyTorch

In [25]:
import torch.nn as nn

In [26]:
# Inputs (temperatura, chuva, umidade)
inputs = np.array(
    [
        [73, 67, 43],
        [91, 88, 64],
        [87, 134, 58],
        [102, 43, 37],
        [69, 96, 70],
        [74, 66, 43],
        [91, 87, 65],
        [88, 134, 59],
        [101, 44, 37],
        [68, 96, 71],
        [73, 66, 44],
        [92, 87, 64],
        [87, 135, 57],
        [103, 43, 36],
        [68, 97, 70],
    ],
    dtype="float32",
)

# Targets (maçãs, laranjas)
targets = np.array(
    [
        [56, 70],
        [81, 101],
        [119, 133],
        [22, 37],
        [103, 119],
        [57, 69],
        [80, 102],
        [118, 132],
        [21, 38],
        [104, 118],
        [57, 69],
        [82, 100],
        [118, 134],
        [20, 38],
        [102, 120],
    ],
    dtype="float32",
)

In [27]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [28]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [29]:
from torch.utils.data import TensorDataset

In [30]:
# Definimos o dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [31]:
# Definimos o DataLoader resposável por gerar os batches
from torch.utils.data import DataLoader

In [32]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

`Shuffle` ajuda a randomizar o lote antes de passar para o modelo. Ele ajuda a melhorar a eficiência do algoritmo de otimização e leva a uma melhora acelerada da perda.

In [33]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  88.,  64.],
        [ 73.,  67.,  43.],
        [103.,  43.,  36.],
        [102.,  43.,  37.],
        [ 68.,  97.,  70.]])
tensor([[ 81., 101.],
        [ 56.,  70.],
        [ 20.,  38.],
        [ 22.,  37.],
        [102., 120.]])


In [34]:
# Definimos o modelo
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3330,  0.4656, -0.3036],
        [ 0.2406,  0.0802, -0.1553]], requires_grad=True)
Parameter containing:
tensor([ 0.3518, -0.0056], requires_grad=True)


O método `.parameters` retorna uma lista das matrizes dos pesos e do bias no modelo.

In [35]:
# Parametros
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3330,  0.4656, -0.3036],
         [ 0.2406,  0.0802, -0.1553]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3518, -0.0056], requires_grad=True)]

In [36]:
# Geramos as predições
preds = model(inputs)
preds

tensor([[42.7986, 16.2518],
        [52.1935, 19.0049],
        [74.1005, 22.6630],
        [43.1032, 22.2363],
        [46.7704, 13.4215],
        [42.6660, 16.4122],
        [51.4243, 18.7694],
        [74.1298, 22.7483],
        [43.2358, 22.0759],
        [46.1338, 13.0256],
        [42.0294, 16.0163],
        [52.0609, 19.1653],
        [74.8697, 22.8985],
        [43.7399, 22.6322],
        [46.9030, 13.2611]], grad_fn=<AddmmBackward0>)

## Função de perda

In [37]:
import torch.nn.functional as F

In [38]:
# Definimos a função de perda
loss_fn = F.mse_loss

In [39]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(3978.8047, grad_fn=<MseLossBackward0>)


## Otimizador

Podemos utilizar otimizadores embutidos no PyTorch em vez de definir manualmente. Para o gradiente de descida, podemos utilizar `optim.SGD`

In [40]:
# Definimos o otimizador
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [41]:
# Definimos a função de treinamento
def fit(num_epochs, model, loss_fn, opt, train_dl):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            # Geramos as predições
            pred = model(xb)
            # Calculamos a perda
            loss = loss_fn(pred, yb)
            # Computamos os gradientes
            loss.backward()
            # Atualizamos os parâmetros usando os gradientes
            opt.step()
            # Zeramos os gradientes
            opt.zero_grad()
        # Imprimimos a perda a cada 10 epocas
        if (epoch + 1) % 10 == 0:
            print(
                "Epoch [{}/{}], Loss: {:.4f}".format(epoch + 1, num_epochs, loss.item())
            )

In [42]:
# Treinamos o modelo por 100 epocas
fit(980, model, loss_fn, opt, train_dl)

Epoch [10/980], Loss: 572.2422
Epoch [20/980], Loss: 241.2220
Epoch [30/980], Loss: 12.5623
Epoch [40/980], Loss: 243.3409
Epoch [50/980], Loss: 66.4738
Epoch [60/980], Loss: 108.7296
Epoch [70/980], Loss: 46.1147
Epoch [80/980], Loss: 99.3079
Epoch [90/980], Loss: 28.0828
Epoch [100/980], Loss: 67.3047
Epoch [110/980], Loss: 38.7118
Epoch [120/980], Loss: 39.8659
Epoch [130/980], Loss: 22.8745
Epoch [140/980], Loss: 43.4749
Epoch [150/980], Loss: 28.8155
Epoch [160/980], Loss: 39.9774
Epoch [170/980], Loss: 33.0573
Epoch [180/980], Loss: 20.0737
Epoch [190/980], Loss: 33.1086
Epoch [200/980], Loss: 8.5484
Epoch [210/980], Loss: 13.6824
Epoch [220/980], Loss: 20.2416
Epoch [230/980], Loss: 18.3254
Epoch [240/980], Loss: 7.8598
Epoch [250/980], Loss: 23.4201
Epoch [260/980], Loss: 14.4729
Epoch [270/980], Loss: 10.9712
Epoch [280/980], Loss: 16.3766
Epoch [290/980], Loss: 12.1737
Epoch [300/980], Loss: 14.4727
Epoch [310/980], Loss: 12.1568
Epoch [320/980], Loss: 12.2224
Epoch [330/980]

In [43]:
# Geramos as predições
preds = model(inputs)
preds

tensor([[ 57.0504,  70.4346],
        [ 81.6696, 100.2610],
        [118.8812, 133.4099],
        [ 20.9936,  37.9197],
        [101.1947, 117.9174],
        [ 55.7939,  69.3501],
        [ 81.4629, 100.3039],
        [119.1373, 133.9831],
        [ 22.2501,  39.0042],
        [102.2445, 119.0448],
        [ 56.8437,  70.4774],
        [ 80.4131,  99.1765],
        [119.0879, 133.3670],
        [ 19.9438,  36.7923],
        [102.4512, 119.0019]], grad_fn=<AddmmBackward0>)

In [44]:
# Comparando com os targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [45]:
# Predizemos novos valores
model(torch.tensor([[75, 63, 44.]]))

# -- Resposta --
# tensor([[53.4258, 67.6156]], grad_fn=<AddmmBackward0>)

# Significa que o modelo prevê 53 maçãs e 67 laranjas, arredondando.

tensor([[53.4712, 67.5010]], grad_fn=<AddmmBackward0>)

In [68]:
model2 = nn.Sequential(
    nn.Linear(3, 4),
    nn.Sigmoid(),
    nn.Linear(4, 2),
    nn.Sigmoid(),
    nn.Linear(2, 2),
)

In [69]:
optimizer = torch.optim.SGD(model2.parameters(), lr=1e-4)

In [70]:
fit(5000, model2, F.mse_loss, optimizer, train_dl)

Epoch [10/5000], Loss: 6825.5259
Epoch [20/5000], Loss: 9764.5771
Epoch [30/5000], Loss: 6742.0459
Epoch [40/5000], Loss: 6686.0259
Epoch [50/5000], Loss: 8116.0249
Epoch [60/5000], Loss: 10204.5039
Epoch [70/5000], Loss: 9455.8799
Epoch [80/5000], Loss: 9337.6172
Epoch [90/5000], Loss: 12326.6982
Epoch [100/5000], Loss: 11336.5801
Epoch [110/5000], Loss: 6776.5259
Epoch [120/5000], Loss: 6893.5537
Epoch [130/5000], Loss: 5362.0591
Epoch [140/5000], Loss: 6854.4297
Epoch [150/5000], Loss: 6078.3486
Epoch [160/5000], Loss: 5124.6064
Epoch [170/5000], Loss: 8617.7246
Epoch [180/5000], Loss: 3577.6250
Epoch [190/5000], Loss: 8998.0996
Epoch [200/5000], Loss: 4158.7769
Epoch [210/5000], Loss: 9490.4229
Epoch [220/5000], Loss: 2558.2302
Epoch [230/5000], Loss: 5334.0186
Epoch [240/5000], Loss: 7112.1235
Epoch [250/5000], Loss: 3554.5122
Epoch [260/5000], Loss: 6866.6196
Epoch [270/5000], Loss: 3684.4187
Epoch [280/5000], Loss: 10446.1904
Epoch [290/5000], Loss: 5547.8613
Epoch [300/5000], L

In [71]:
print(model2(torch.tensor([[75, 63, 44.]])))
print(targets)

tensor([[75.1097, 90.9135]], grad_fn=<AddmmBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])
